In [12]:
import sys
sys.path.append('../../lib/python3.5/site-packages')
import re
import jieba.analyse
import numpy as np
import jieba
jieba.set_dictionary('dict.txt.big')
jieba.analyse.set_stop_words("./ch_dont_want_word")

In [16]:
from elasticsearch import Elasticsearch
import elasticsearch.helpers
from datetime import datetime

es = Elasticsearch([{'host': '192.168.2.23', 'port': 9200}])

In [17]:
# es.get(index="pixnet", doc_type='food', id=2)

search_body = \
{\
  "_source": ["title","content","tags"], \
  "query" : {\
    "bool": {\
      "must": [\
        { \
          "match" : { \
            "content" : "台中"\
          }\
        },\
        { \
          "match" : { \
            "content" : "早午餐"\
          }\
        },\
#         { \
#           "match" : { \
#             "tags" : "夜"\
#           }\
#         }\
      ],\
#       "filter": \
#         [\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "新竹"\
#               }\
#             }\
#           },\
#           {\
#             "term": {\
#               "tags.keyword": {\
#                 "value": "宵夜"\
#               }\
#             }\
#           }\
#         ]\
    }\
  }\
}
res = es.search(index="pixnet", doc_type="food", body= search_body)
print("%d documents found" % res['hits']['total'])
for i, match in enumerate(res['hits']['hits']):
    print("Top.{} Match\nScore : {}\nTitle : {}\nContent : {}\n\n".format(i, match['_score'], match['_source']['title'], match['_source']['content']))

1094 documents found
Top.0 Match
Score : 14.739791
Title : 【台中早午餐鬆餅】【田樂】老宅新生~~輕鬆吃田樂小公園店!
Content :  
｜店家資訊｜
田樂●小公園店  (for Farm Burger/田樂二店)
地址：台中市西區大忠南街82巷36號
官方網站 ：http://forfarmburger.blogspot.tw/
電話：04-23728672
 因為Pull沒辦法一一追蹤店家的現況喔！
建議大家可以先打電話確認一下^0^
 
話說上次本來要吃田樂一店的貪吃母女檔沒有吃到田樂
反而發現祕密廚房開分店了~就是...《小祕密廚房》
然後就在那兒度過了美好的中午!
不過回到家後...Pull媽還是心繫田樂
一直對著Pull轟炸說她要吃田樂她要吃田樂!!!!!
於是Pull想說一定要找一天和Pull媽去田樂不然她會哭哭~~~
剛好某天體驗完胖達之後
肚子還有點空虛...
(不是真的餓~是心靈上的空虛啦)
結果腦中突然想到了田樂!
於是我們倆就決議拜訪田樂小公園店(田樂二店)
(因為上次雖然沒吃到但有看到一店的用餐環境...好擁擠)
於是就開啟了『Pull和Pull媽美食吃不停之田樂二店的奇幻旅程』
 

田樂小公園店(以下簡稱田樂)位在大忠南街82巷
因為在很隱密的巷弄內
所以要很仔細地尋找喔!不然會忽略掉...
走進82巷後再往裡面一直走
就會發現一個比較老舊的社區
田樂就在最顯眼的那個角角了!
Pull覺得最近老宅改建/翻修的案例真的很多耶
而且也都成功製造了話題喔
像是田樂最近在網路上可說是紅翻天啊
於是乎也造成了上次我們去田樂一店卻吃不到的悲劇
這次其實我們也不抱很多希望...
因為一店就滿成這樣了
那二店豈不是更恐怖~~~
只是想說碰個運氣看看會不會剛好被我們訂到位!
  
 田樂的外觀就是老宅囉!
這個木牌蠻酷的欸
真的有很舊很舊的感覺~好藝術!
  
 外面有種花花草草...真是賞心悅目!
窗戶透露著黃光的溫暖氣息
  
 總是免不了要放小立板在外頭!
田樂二店在試賣期間只有販賣鬆餅&朝食...
但最近解除試賣期所以也開始販賣起在一店很夯的漢堡系列!
  
看完上面的照片後大家有沒有發現到一件很古怪的事...
為什麼田樂一樓沒有大門之類的東西...
這樣是要怎麼進去呢(兩手一攤~)